In [1]:
# privateとpublicの区別は`Base.propertynames`の定義で行える。
@doc propertynames

```
propertynames(x, private=false)
```

Get a tuple or a vector of the properties (`x.property`) of an object `x`. This is typically the same as [`fieldnames(typeof(x))`](@ref), but types that overload [`getproperty`](@ref) should generally overload `propertynames` as well to get the properties of an instance of the type.

`propertynames(x)` may return only "public" property names that are part of the documented interface of `x`.   If you want it to also return "private" fieldnames intended for internal use, pass `true` for the optional second argument. REPL tab completion on `x.` shows only the `private=false` properties.

See also: [`hasproperty`](@ref), [`hasfield`](@ref).


In [2]:
module O

"""
抽象平面ベクトルの型
"""
abstract type AbstractPlanarVector{T} end

"""タプル版和集合(重複を削除)"""
tupleunion(x) = Tuple(unique(x))
tupleunion(x, y) = Tuple(unique((x..., y...)))
tupleunion(x, y, z...) = tupleunion(tupleunion(x, y), z...)

"""
抽象平面ベクトルのpublic propertiesは x と y

タブ補完でもx, yしか表示されなくなる。
"""
function Base.propertynames(p::AbstractPlanarVector, private=false)
    public_properties = (:x, :y)
    private && return tupleunion(public_properties, fieldnames(p))
    public_properties
end

"""
抽象平面ベクトルは (x, y) の形式で表示される
"""
Base.show(io::IO, p::AbstractPlanarVector) = show(io, (p.x, p.y))

"""
AbstractPlanarVector{T}型の抽象平面ベクトルの成分の型は T
"""
Base.eltype(x::AbstractPlanarVector{T}) where T = T

"""
平面ベクトルの型

x, yをfieldsに持ち、それらがそのままpublic propertiesになる。
"""
struct PlanarVector{T} <: AbstractPlanarVector{T} x::T; y::T end

"""
標準基底の型

x, yをfieldsとして持たず、型のパラメータ i で何番目の基底ベクトルであるかを指定。
"""
struct CanonBasis{T, i} <: AbstractPlanarVector{T} end
function CanonBasis{T}(i) where T
    @assert i == 1 || i == 2
    CanonBasis{T, i}()
end
CanonBasis(i) = CanonBasis{Int}(i)

"""
標準基底のpublic propertiesの取得メソッド

x, yをfieldsとして持たないので定義してやる必要がある。

* i == 1 ⇒ (x, y) = (1, 0)
* i == 2 ⇒ (x, y) = (0, 1)
"""
function Base.getproperty(::CanonBasis{T, i}, name::Symbol) where {T, i}
    name === :x && return ifelse(i == 1, one(T), zero(T))
    name === :y && return ifelse(i == 2, one(T), zero(T))
    error("type CanonBasis has no public property $name")
end

########## ベクトルの演算達

# 抽象平面ベクトル p, q について、 +p, -p, p + q, p - q を定義
for op in (:+, :-)
    @eval Base.$op(p::AbstractPlanarVector) = PlanarVector($op(p.x), $op(p.y))
    @eval Base.$op(p::AbstractPlanarVector, q::AbstractPlanarVector) =
        PlanarVector($op(p.x, q.x), $op(p.y, q.y))
end

# 抽象平面ベクトル p とスカラー a について、 a * p, p * a, a \ p, p / a を定義
Base.:*(a, p::AbstractPlanarVector) = PlanarVector(a * p.x, a * p.y)
Base.:*(p::AbstractPlanarVector, a) = PlanarVector(p.x * a, p.y * a)
Base.:\(a, p::AbstractPlanarVector) = PlanarVector(a \ p.x, a \ p.y)
Base.:/(p::AbstractPlanarVector, a) = PlanarVector(p.x / a, p.y / a)

# LinearAlgebra の dot の定義の準備
using LinearAlgebra

##### 抽象ベクトルの内積を定義

# まず、一般的な定義式で内積を定義
LinearAlgebra.dot(p::AbstractPlanarVector, q::AbstractPlanarVector) =
    conj(p.x) * q.x + conj(p.y) * q.y

# 次に、標準基底の場合に特殊化した定義を行う。多重ディスパッチを本質的に使っている。
LinearAlgebra.dot(p::CanonBasis{T, i}, q::AbstractPlanarVector{U}) where {T, i, U} =
    (P = promote_type(T, U); P(i == 1 ? q.x : q.y))
LinearAlgebra.dot(p::AbstractPlanarVector{U}, q::CanonBasis{T, i}) where {U, T, i} =
    (P = promote_type(T, U); P(conj(i == 1 ? p.x : p.y)))

# Juliaの多重ディスパッチでは上の2つの場合のintersectionの場合の定義もしておく必要がある。
LinearAlgebra.dot(p::CanonBasis{T, i}, q::CanonBasis{U, j}) where {T, i, U, j} =
    (P = promote_type(T, U); i == j ? one(P) : zero(P))

end

Main.O

In [3]:
p = O.PlanarVector(2+im, 3+im)

(2 + 1im, 3 + 1im)

In [4]:
q = O.PlanarVector(-5.0, 10.0)

(-5.0, 10.0)

In [5]:
p + q

(-3.0 + 1.0im, 13.0 + 1.0im)

In [6]:
eltype(p + q)

ComplexF64 (alias for Complex{Float64})

In [7]:
-3p + 4q

(-26.0 - 3.0im, 31.0 - 3.0im)

In [8]:
O.dot(p, q)

20.0 - 5.0im

In [9]:
e1 = O.CanonBasis{Float64}(1)

(1.0, 0.0)

In [10]:
e2 = O.CanonBasis(2)

(0, 1)

In [11]:
O.dot(e1, e2)

0.0

In [12]:
O.dot(O.CanonBasis{ComplexF64}(1), e1)

1.0 + 0.0im

In [13]:
e1 - e2

(1.0, -1.0)

In [14]:
3*e1 - e2/2

(3.0, -0.5)

In [15]:
p.x, p.y

(2 + 1im, 3 + 1im)

In [16]:
e1.x, e1.y

(1.0, 0.0)

In [17]:
getx(p::O.AbstractPlanarVector) = p.x
gety(p::O.AbstractPlanarVector) = p.y

gety (generic function with 1 method)

In [18]:
getx(p), gety(p)

(2 + 1im, 3 + 1im)

In [19]:
@code_typed getx(p)

CodeInfo(
1 ─ %1 = Base.getfield(p, :x)::Complex{Int64}
└──      return %1
) => Complex{Int64}

In [20]:
@code_typed gety(q)

CodeInfo(
1 ─ %1 = Base.getfield(p, :y)::Float64
└──      return %1
) => Float64

In [21]:
@code_typed getx(e2)

CodeInfo(
1 ─     return 0
) => Int64

In [22]:
@code_typed gety(e2)

CodeInfo(
1 ─     return 1
) => Int64

In [23]:
@code_typed O.dot(p, q)

CodeInfo(
1 ─ %1  = Base.getfield(p, :x)::Complex{Int64}
│   %2  = Base.getfield(%1, :re)::Int64
│   %3  = Base.getfield(%1, :im)::Int64
│   %4  = Base.neg_int(%3)::Int64
│   %5  = Base.getfield(q, :x)::Float64
│   %6  = Base.sitofp(Float64, %2)::Float64
│   %7  = Base.mul_float(%5, %6)::Float64
│   %8  = Base.sitofp(Float64, %4)::Float64
│   %9  = Base.mul_float(%5, %8)::Float64
│   %10 = Base.getfield(p, :y)::Complex{Int64}
│   %11 = Base.getfield(%10, :re)::Int64
│   %12 = Base.getfield(%10, :im)::Int64
│   %13 = Base.neg_int(%12)::Int64
│   %14 = Base.getfield(q, :y)::Float64
│   %15 = Base.sitofp(Float64, %11)::Float64
│   %16 = Base.mul_float(%14, %15)::Float64
│   %17 = Base.sitofp(Float64, %13)::Float64
│   %18 = Base.mul_float(%14, %17)::Float64
│   %19 = Base.add_float(%7, %16)::Float64
│   %20 = Base.add_float(%9, %18)::Float64
│   %21 = %new(ComplexF64, %19, %20)::ComplexF64
└──       return %21
) => ComplexF64

In [24]:
@code_typed O.dot(e1, q)

CodeInfo(
1 ─ %1 = Base.getfield(q, :x)::Float64
└──      return %1
) => Float64

In [25]:
@code_typed O.dot(p, e2)

CodeInfo(
1 ─      goto #3 if not false
2 ─      nothing::Nothing
3 ┄ %3 = Base.getfield(p, :y)::Complex{Int64}
│   %4 = Base.getfield(%3, :re)::Int64
│   %5 = Base.getfield(%3, :im)::Int64
│   %6 = Base.neg_int(%5)::Int64
│   %7 = %new(Complex{Int64}, %4, %6)::Complex{Int64}
└──      return %7
) => Complex{Int64}

In [26]:
@code_llvm debuginfo=:none O.dot(p, e2)

; Function Attrs: uwtable
define void @julia_dot_2208([2 x i64]* noalias nocapture sret([2 x i64]) %0, [2 x [2 x i64]]* nocapture nonnull readonly align 8 dereferenceable(32) %1) #0 {
top:
  %2 = getelementptr inbounds [2 x [2 x i64]], [2 x [2 x i64]]* %1, i64 0, i64 1, i64 0
  %3 = getelementptr inbounds [2 x [2 x i64]], [2 x [2 x i64]]* %1, i64 0, i64 1, i64 1
  %4 = load i64, i64* %3, align 8
  %5 = sub i64 0, %4
  %6 = load i64, i64* %2, align 8
  %.sroa.0.0..sroa_idx = getelementptr inbounds [2 x i64], [2 x i64]* %0, i64 0, i64 0
  store i64 %6, i64* %.sroa.0.0..sroa_idx, align 8
  %.sroa.2.0..sroa_idx1 = getelementptr inbounds [2 x i64], [2 x i64]* %0, i64 0, i64 1
  store i64 %5, i64* %.sroa.2.0..sroa_idx1, align 8
  ret void
}


In [27]:
@code_typed O.dot(e2, e2)

CodeInfo(
1 ─     return 1
) => Int64

In [28]:
@code_typed O.dot(e1, e2)

CodeInfo(
1 ─     goto #3 if not false
2 ─     nothing::Nothing
3 ┄     return 0.0
) => Float64

In [29]:
@code_llvm debuginfo=:none O.dot(e1, e2)

; Function Attrs: uwtable
define double @julia_dot_2240() #0 {
top:
  ret double 0.000000e+00
}


In [30]:
using BenchmarkTools
n = 10^6
v = [O.PlanarVector(randn(2)...) for _ in 1:n]
a = O.PlanarVector(1, 0)
e = O.CanonBasis(1)

@show sum(Base.Fix1(O.dot, a), v) == sum(Base.Fix1(O.dot, e), v)
@btime sum($(Base.Fix1(O.dot, a)), $v)
@btime sum($(Base.Fix1(O.dot, e)), $v)

sum(Base.Fix1(O.dot, a), v) == sum(Base.Fix1(O.dot, e), v) = true
  691.900 μs (0 allocations: 0 bytes)
  640.600 μs (0 allocations: 0 bytes)


-911.6979226337576

In [31]:
@code_native debuginfo=:none O.dot(a, q)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	vcvtsi2sdq	(%rcx), %xmm0, %xmm0
	vmulsd	(%rdx), %xmm0, %xmm0
	vcvtsi2sdq	8(%rcx), %xmm1, %xmm1
	vmulsd	8(%rdx), %xmm1, %xmm1
	vaddsd	%xmm1, %xmm0, %xmm0
	popq	%rbp
	retq
	nop


In [32]:
@code_native debuginfo=:none O.dot(e, q)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	vmovsd	(%rcx), %xmm0                   # xmm0 = mem[0],zero
	popq	%rbp
	retq
	nopw	(%rax,%rax)
